# Dataset

Precios de casas:

https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('data/house-prices-advanced-regression-techniques/train.csv',index_col=0)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

## Drop duplicates

In [ ]:
df = df.drop_duplicates()
df.shape

## Drop NA

In [ ]:
# Borra todas las filas con NaN
df.dropna()

In [ ]:
columns_na = df.isna().sum().sort_values(ascending = False)
(columns_na[columns_na>0]/len(df))*100

Todas las filas tienen al menos un valor nulo, por lo tanto se borra todo el dataframe.<br>
Intentemos borrar columnas

In [ ]:
df_drop_1 = df.dropna(axis=1)
print(df_drop_1.shape)
df_drop_1.head()

## ¿Que eliminamos?
### Analizando la columna **PoolQC**

Según **data_description.txt**


PoolQC: Pool quality

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool
       
Borramos información vital en el DataFrame.

En lugar de borrarla la podemos convertir en  una columna de datos  ordinales.

In [ ]:
df['PoolQC'].unique()

In [ ]:
poolQCvalues={'Ex':4,'Gd':3,'TA':2,'Fa':1,np.nan:0}
df['PoolQC'] = df['PoolQC'].map(poolQCvalues)

In [ ]:
print('Numero de NaN en la columna:', df['PoolQC'].isna().sum())
df['PoolQC'].sample(5)

In [ ]:
df['PoolQC'].unique()

### Analizando la columna **CentralAir**

CentralAir: Central air conditioning

       N	No
       Y	Yes
	

In [ ]:
df['CentralAir'].unique()

In [ ]:
centralAirvalues={'N' : 0,'Y':1}
df['CentralAir'] = df['CentralAir'].map(centralAirvalues)

In [ ]:
df['CentralAir'].sample(5)

### Electrical

Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed

In [ ]:
print('Numero de NaN en la columna:', df['Electrical'].isna().sum())

In [ ]:
mode = df['Electrical'].mode()[0]
print(mode)

In [ ]:
df['Electrical'] = df['Electrical'].fillna(mode)

In [ ]:
print('Numero de NaN en la columna:', df['Electrical'].isna().sum())

In [ ]:
df['Electrical'].unique()

In [ ]:
df['Electrical'].head()

In [ ]:
df['Electrical'] = df['Electrical'].astype('category').cat.codes

In [ ]:
df['Electrical'].head()

No existe una relación de orden en la columna **Electrical**, por lo tanto estas  categorias no tiene mucho sentido analizarlo como una variable ordinal, deberiamos mejor convertirlas a **Dummy Variables**.

# Cuartiles y  datos atipicos

### Limpieza por cuartiles

Los cuartiles son valores que dividen una muestra de datos en cuatro partes iguales. Con los cuartiles se puede evaluar rápidamente la dispersión y la tendencia central de los datos.

El recuadro se extiende desde los valores de cuartil Q1 a Q3 de los datos, con una línea en la mediana (Q2). Los **bigotes** se extienden desde los bordes de la **caja** para mostrar el rango de los datos. 
La posición de los **bigotes** se establece de forma predeterminada en 1.5 * IQR (IQR = Q3 - Q1) desde los bordes de la caja. Los puntos atípicos son aquellos que se encuentran más allá del final de los bigotes.

In [ ]:
%matplotlib inline
df.boxplot('SalePrice')

In [ ]:
sP = df['SalePrice']
IQR = 1.5*(sP.quantile(.75)-sP.quantile(.25))
lim_sup = sP.quantile(.75)+IQR
lim_inf = sP.quantile(.25)-IQR

In [ ]:
sP_clean = sP[(sP >= lim_sup) | (sP <= lim_inf)]
sP_clean

In [ ]:
df.drop(sP_clean.index)

### Limpieza por desviación estandar

La desviación estándar es una de las medidas de estadística mas populares y de mayor uso. Esta medida indica el grado de dispersión alrededor de la media. Una desviación grande indica datos muy dispersos y una desviación pequeña indica un menor grado de dispersión.

El puntaje Z (**Z -score**) es el número de desviaciones estándar por las cuales el valor de una observación está por encima del valor medio de lo que se está observando.

In [ ]:
from scipy import stats
sP = df['SalePrice']
z = np.abs(stats.zscore(sP))
sP_clean = sP[(z>3) | (z<-3)]
df.drop(sP_clean.index)